In [3]:
import ee
import matplotlib.pyplot as plt
import csv
import datetime

# Inicializar a API do Earth Engine
ee.Initialize()

# Definir o polígono de interesse
polygon = ee.Geometry.Polygon(
    [[[-46.70, -23.60], [-46.65, -23.60], [-46.65, -23.65], [-46.70, -23.65], [-46.70, -23.60]]]
)

# Definir as datas de início e fim do período
start_date = '2023-01-01'
end_date = '2023-05-31'

# Filtrar as imagens Sentinel-2
collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(polygon) \
    .filterDate(start_date, end_date) \
    .select(['B4', 'B8'])

# Calcular o NDVI para cada imagem
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

collection_with_ndvi = collection.map(calculate_ndvi)

# Exportar a coleção como uma tabela
export_table = ee.batch.Export.table.toDrive(
    collection=collection_with_ndvi,
    description='NDVI_timeseries',
    folder='GEE_exports',
    fileFormat='CSV'
)
export_table.start()

# Esperar até a exportação ser concluída
while export_table.active():
    print('Exporting...')

# Ler o arquivo CSV exportado
csv_path = 'GEE_exports/NDVI_timeseries.csv'
ndvi_values = []
dates = []

with open(csv_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        ndvi_values.append(float(row['NDVI']))
        dates.append(row['system:index'])

# Converter as datas para objetos datetime
dates = [datetime.datetime.strptime(date, '%Y%m%dT%H%M%S') for date in dates]

# Plotar o gráfico de linhas
plt.plot(dates, ndvi_values)
plt.xlabel('Data')
plt.ylabel('NDVI')
plt.title('Série Temporal de NDVI')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...


FileNotFoundError: [Errno 2] No such file or directory: 'GEE_exports/NDVI_timeseries.csv'